In [1]:
# import all the things
import pandas as pd
from sqlalchemy import create_engine

In [14]:
# path to Golden Globes csv file
gg_file = "Resources/golden_globe_awards.csv"

# read in Golden Globes csv file
gg_df = pd.read_csv(gg_file)
gg_df.head()

,year_film,year_award,ceremony,category,nominee,film,win
0,1943,1944,1,Best Performance by an Actress in a Supporting...,Katina Paxinou,For Whom The Bell Tolls,True
1,1943,1944,1,Best Performance by an Actor in a Supporting R...,Akim Tamiroff,For Whom The Bell Tolls,True
2,1943,1944,1,Best Director - Motion Picture,Henry King,The Song Of Bernadette,True
3,1943,1944,1,Picture,The Song Of Bernadette,NaN,True
4,1943,1944,1,Actress In A Leading Role,Jennifer Jones,The Song Of Bernadette,True


In [15]:
# drop unnecessary columns ceremony and year_film
new_gg_df = gg_df.drop(columns={"ceremony","year_film"})
new_gg_df.head()

,year_award,category,nominee,film,win
0,1944,Best Performance by an Actress in a Supporting...,Katina Paxinou,For Whom The Bell Tolls,True
1,1944,Best Performance by an Actor in a Supporting R...,Akim Tamiroff,For Whom The Bell Tolls,True
2,1944,Best Director - Motion Picture,Henry King,The Song Of Bernadette,True
3,1944,Picture,The Song Of Bernadette,NaN,True
4,1944,Actress In A Leading Role,Jennifer Jones,The Song Of Bernadette,True


In [96]:
# remove non movie based categories
movies_only = new_gg_df[new_gg_df['category'].str.contains('Actor') == False]
movies_only = movies_only[movies_only['category'].str.contains('Actress') == False]
movies_only = movies_only[movies_only['category'].str.contains('Director') == False]
movies_only = movies_only[movies_only['category'].str.contains('International') == False]
movies_only = movies_only[movies_only['category'].str.contains('Special') == False]
movies_only = movies_only[movies_only['category'].str.contains('Screenplay') == False]
movies_only = movies_only[movies_only['category'].str.contains('Song') == False]
movies_only = movies_only[movies_only['category'].str.contains('Score') == False]
movies_only = movies_only[movies_only['category'].str.contains('Cinematography') == False]
movies_only = movies_only[movies_only['category'].str.contains('Foreign') == False]
movies_only = movies_only[movies_only['category'].str.contains('Star') == False]
movies_only = movies_only[movies_only['category'].str.contains('Juvenile') == False]
movies_only = movies_only[movies_only['category'].str.contains('International') == False]
movies_only = movies_only[movies_only['category'].str.contains('Color') == False]
movies_only = movies_only[movies_only['category'].str.contains('Henrietta') == False]
movies_only = movies_only[movies_only['category'].str.contains('Cecil') == False]
movies_only = movies_only[movies_only['category'].str.contains('International') == False]
movies_only = movies_only[movies_only['category'].str.contains('Documentary') == False]
movies_only = movies_only[movies_only['category'].str.contains('Television') == False]
movies_only = movies_only[movies_only['category'].str.contains('Hollywood') == False]


# drop film column and rename nominee column to film
movies_only = movies_only.drop(columns="film")
movies_only = movies_only.rename(columns={"nominee":"film"})

In [98]:
movies_only.head(50)

,year_award,category,film,win
3,1944,Picture,The Song Of Bernadette,True
9,1945,Picture,Going My Way,True
16,1946,Picture,The Lost Weekend,True
24,1947,Picture,The Best Years Of Our Lives,True
35,1948,Picture,Gentleman's Agreement,True
46,1949,Picture,Johnny Belinda,True
47,1949,Picture,The Treasure of the Sierra Madre,True
73,1950,Picture,All The King's Men,True
74,1950,Picture,Come To The Stable,False
112,1951,Picture,Sunset Boulevard,True


In [ ]:
# need to fix all movies that are formatted "xxxx, The" to "The xxxx" 
#ie Lord of the Rings, The needs to be The Lord of the Rings